In [ ]:
import os
import configparser
import pandas as pd

os.environ['KMP_DUPLICATE_LIB_OK']='True'
%matplotlib inline

### Config

In [ ]:
config = configparser.ConfigParser()
config.read('settings.ini');

In [ ]:
train_path = config['Path']['train_path']
test_path = config['Path']['test_path']
models_path = config['Path']['models_path']
metadata_path = config['Path']['metadata_path']
predictions_path  = config['Path']['predictions_path']
other_class = config['Class']['other']
classes  = [other_class] + config['Class']['classes'].split()

labeling_model = config['Labeling']['labeling_model'] 
labeling_threshold = float(config['Labeling']['labeling_threshold'])

learning_rate = float(config['Training']['learning_rate'])
bs_per_image = int(config['Training']['bs_per_image'])
max_iter = int(config['Training']['max_iter'])
threshold = float(config['Training']['threshold'])
model_config = config['Training']['model_config']

In [ ]:
metadata_train_path = os.path.join(metadata_path, 'metadata_train.csv')
metadata_test_path = os.path.join(metadata_path, 'metadata_test.csv')

### Metadata

In [ ]:
from metadata import create_metadata
from detector import Detector

In [ ]:
#detector = Detector(model_config, 'animals_train_full', learning_rate, max_iter, bs_per_image, len(classes), threshold)
#detector.load_weights('output/{}/model_final.pth'.format(labeling_model))
#detector.set_threshold(labeling_threshold)

In [ ]:
create_metadata() #create_metadata(detector)

In [ ]:
metadata_train = pd.read_csv(metadata_train_path, index_col='id')
metadata_test = pd.read_csv(metadata_test_path, index_col='id')

In [ ]:
metadata_train = metadata_train.sample(frac=1).reset_index(drop=True)
metadata_test = metadata_test.sample(frac=1).reset_index(drop=True)

In [ ]:
metadata_train.head()

# Detection

In [ ]:
from detector import get_animals_dicts, get_animals_dicts_test, init_catalog

In [ ]:
test_val_split = float(config['Training']['test_val_split'])
test_val_split_n = int(len(metadata_train) * test_val_split)

In [ ]:
init_catalog('animals_train_full', metadata_train, train_path, classes)
init_catalog('animals_train', metadata_train[:test_val_split_n], train_path, classes)    
init_catalog('animals_val', metadata_train[test_val_split_n:], train_path, classes)    
init_catalog('animals_test', metadata_test, test_path, classes, train=False)     

### Training

In [ ]:
from detector import Detector

In [ ]:
detector = Detector(model_config, 'animals_train_full', learning_rate, max_iter, bs_per_image, len(classes), threshold,
                   model_name='model_c')

In [ ]:
#detector.load_weights('output/last/model_final.pth')

In [ ]:
detector.train()

### Prediction

In [ ]:
import random
import matplotlib.pyplot as plt
import cv2
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode

In [ ]:
detector.set_threshold(0.35)

In [ ]:
dataset_dicts = get_animals_dicts_test(metadata_test, test_path)

In [ ]:
for d in random.sample(dataset_dicts, 10):    
    im = cv2.imread(d["file_name"])
    outputs = detector.predict(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=detector.metadata, 
                   scale=0.8,
                   instance_mode=ColorMode.IMAGE   
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    fig, ax = plt.subplots(1, 1, figsize=(15, 15))
    ax.imshow(v.get_image())
    plt.show()